In [28]:
import torch
import torch.nn as nn
import numpy as np

class Net(nn.Module):
    def __init__(self,  state_size,  action_size, frames, hidden, layers,
                 lstm1_w, lstm2_w, fc_w):
        super(Net, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.frames = frames
        self.hidden = hidden
        self.layers = layers
        self.lstm = nn.LSTM(self.state_size,
                            self.hidden,
                            self.layers,
                            bidirectional=False,
                            bias=False)

        self.head = nn.Linear(self.hidden, self.action_size, bias=False)
        self.lstm.weight_ih_l0.data = lstm1_w
        self.lstm.weight_hh_l0.data = lstm2_w
        self.head.weight.data = fc_w

    def forward(self, state, h):
        x, h = self.lstm(state, h)
        x = x[:, -1]
        x = F.softmax(self.head(x), dim=-1), h
        return x

state_size = 32
action_size = 5
hidden_size = 32
layers = 1
frames = 4
fs = state_size*frames
ls = (fs * hidden_size)
fcs = action_size * hidden_size
genome_size = ls*2 + fcs
genome = np.random.uniform(-1, 1, genome_size)
lstm1_w = torch.Tensor(np.reshape(genome[:ls], (fs, state_size)))
lstm2_w = torch.Tensor(np.reshape(genome[ls:ls*2], (fs, state_size)))
fc_w = torch.Tensor(np.reshape(genome[ls*2:], (action_size, hidden_size)))
net = Net(state_size, action_size, frames, hidden_size, layers, lstm1_w, lstm2_w, fc_w)
print(net)

Net(
  (lstm): LSTM(32, 32, bias=False)
  (head): Linear(in_features=32, out_features=5, bias=False)
)


In [12]:
import numpy as np
import pickle

def save_genomes(genome_pool):
    with open("genome_pool.pkl", "wb") as f:
        f.write(pickle.dumps(genome_pool))

def load_genomes():
    n = []
    with open("genome_pool.pkl", "rb") as f:
        n = pickle.load(f)
    return n

genome_pool = []
pool_size = 1000

for _ in range(pool_size):
    genome = np.random.uniform(-1, 1, genome_size)
    fitness = None
    genome_pool.append([genome, fitness])

save_genomes(genome_pool)

new_genome_pool = load_genomes()
print(len(new_genome_pool))

1000


1000
